# Assignment 1

## Conceptual Questions

### Markov Assumption
It is based on the idea that the future state of a system depends only on its present state. In easier words, what happens next only depends on what situation you are right now.

An example could be robot docking to the charger. The robot cares only on its current position with respect to the charger, instead of taking into account all the previous positions.

### Markov Decision Process
In an MDP, we have a decision maker, called an agent, that interacts with the environment. The interactions occur sequentially over time. 

#### Components of RL used in MDP:
- Agents (A) are the central concept of the field of RL. 
- Environment (E) is a well-defined and structured system that agents interact with over time.
- States (S) provide information about current context of the environment. States represent different situations, configurations, or conditions agent can be in.
- Actions (A) are decisions made by the agent to influance the stae of the environment. There are a set of permissible actions that the RL agent can takel
- Rewards (R) is a numerical signal the agent receieves after each action. The goal of the model is to maximize the overall reward.
- Termination Condition: end of episode based on reaching a particular state, a certain number of time steps, or some other scriteria.
- Observations are feedback from the environment to the agent. 


### Markov Reward Process
A simplified version of MDP where there are no actions to choose from. 
It is defin4ed by the following components:
    1. States (s)
    2. Transition Probabilities (P)
    3. Reward Function (R)
    4. Discount Factor ($\gamma$)

In order to reduce MDP to MRP, we fix the policy $\pi$ and replace the action-based transition with a state-only transition.

### Policy Definition
A policy is a set of instructions that tells an agent what action to take in each state. 
- Deterministic Policy is a type of policy that always chooses the same action for a given state.
- Stochastic Policy is a type of policy that chooses randomly according to the probability distribution over actions. 


### Infinite Horizon Value Function
Its goal is to computye the total expected reward the agent receives over an infinte number of steps into the futuer. 
The discount factor $\gamma \in [0, 1)$ controls how much future rewards are worth compared to immediate rewards. 
- if $\gamma$ is close to 0, the agent focuses on immediate rewards
- if $\gamma$ is close to 1, the agent values future rewards more

If $\gamma \ge 1$, the function most probably will diverge resulting the grow of uncontrollability. 


### Policy Iteration
It is an algorithm that is used to find optimal policy in MDP

## Analytical Problems

1. Value Function Computation:

$$S = \{A, B\}$$

$$\gamma = 0.9$$

$$
P = \begin{bmatrix}
0.5 & 0.5 \\
0.4 & 0.6
\end{bmatrix}
$$

$$
R = \begin{bmatrix}
5 \\
2 
\end{bmatrix}
$$

Solution:

$$ V = R + \gamma PV 
$$

$$ (I - \gamma P) V = R $$


$$ V_A = 5 + 0.9(0.5V_A + 0.5V_B) = 5 + 0.45V_A + 0.45V_B $$
$$ V_B = 2 + 0.9(0.4V_A + 0.6V_B) = 2 + 0.36V_A + 0.54V_B $$


$$
\begin{bmatrix}
-0.55 & -0.45 \\
-0.36 & -0.46
\end{bmatrix}
\begin{bmatrix}
V_A\\
V_B
\end{bmatrix} = 
\begin{bmatrix}
5\\
2
\end{bmatrix}
$$

$$ V_A = 35.16 \quad V_B = 31.86$$

In [8]:
import numpy as np
gamma = 0.9

R = np.array([5, 2])

P = np.array([
    [0.5, 0.5],
    [0.4, 0.6]
])

V = np.array([0.0, 0.0])

for k in range(3):
    V = R + gamma * P.dot(V)
    print(f"Iteration {k + 1}: {V}")

Iteration 1: [5. 2.]
Iteration 2: [8.15 4.88]
Iteration 3: [10.8635  7.5692]


In [14]:
import json
import numpy as np

def load_mdp_data(f_dir):
    with open(f_dir, "r") as f:
        return json.load(f)


def init_rand_policy(states, actions):
    policy = {s: np.random.choice(actions) for s in states}
    return policy

def policy_evaluation(policy, transitions, states, gamma, theta=1e-6):
    V = {s: 0 for s in states}
    while True:
        delta = 0
        for s in states:
            a = policy[s]
            v_new = 0
            for trans in transitions[str(s)][a]:
                ns = trans["next_state"]
                prob = trans["prob"]
                reward = trans["reward"]
                v_new += prob * (reward + gamma * V[ns])
            delta = max(delta, abs(v_new - V[s]))
            V[s] = v_new
        if delta < theta:
            break
    return V


In [19]:
def policy_improvement(V, states, actions, transitions, gamma, policy):
    policy_stable = True
    for s in states:
        old_action = policy[s]
        action_values = {}
        for a in actions:
            v = 0
            for trans in transitions[str(s)][a]:
                ns = trans["next_state"]
                prob = trans["prob"]
                reward = trans["reward"]
                v += prob * (reward + gamma * V[ns])
            action_values[a] = v
        best_action = max(action_values, key=action_values.get)
        policy[s] = best_action
        if best_action != old_action:
            policy_stable = False
    return policy_stable, policy

In [20]:
def policy_iteration(mdp_file):
    mdp_data = load_mdp_data(mdp_file)
    states = mdp_data["states"]
    actions = mdp_data["actions"]
    gamma = mdp_data["discount_factor"]
    transitions = mdp_data["transitions"]

    policy = init_rand_policy(states, actions)
    print("rand policy:", policy)

    iteration = 0
    while True:
        iteration += 1
        print(f"\nPolicy Iteration: {iteration}")
        V = policy_evaluation(policy, transitions, states, gamma)
        stable, policy = policy_improvement(V, states, actions, transitions, gamma, policy)
        print(f"Policy after iteration {iteration}: {policy}")
        if stable:
            break

    print("\nOptimal policy:", policy)
    print("Optimal value function:")
    for s in states:
        print(f"V({s}) = {V[s]:.4f}")

    return policy, V

In [21]:
mdp_file = "assignments/mdp_data.json"
policy_iteration(mdp_file)

rand policy: {0: np.str_('right'), 1: np.str_('left'), 2: np.str_('left')}

Policy Iteration: 1
Policy after iteration 1: {0: 'left', 1: 'right', 2: 'left'}

Policy Iteration: 2
Policy after iteration 2: {0: 'left', 1: 'right', 2: 'left'}

Optimal policy: {0: 'left', 1: 'right', 2: 'left'}
Optimal value function:
V(0) = 14.3523
V(1) = 14.4828
V(2) = 12.7586


({0: 'left', 1: 'right', 2: 'left'},
 {0: 14.352277771003633, 1: 14.482753998358886, 2: 12.758616298440941})